In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Sparse Attention is Consistent with Feature Importance

A simple notebook for reproducing our results.

In [ ]:
from importlib import reload
import logging
from ane_research.config import Config
from ane_research.utils.logger import initialize_logger
from IPython.display import Image, display

# clear any logging relics
reload(logging)
logger = initialize_logger(Config.logger_name, in_notebook=True)

## Option 1: Run All Experiments

In [ ]:
from ane_research.utils.experiments import run_all_experiments_in_dir

experiments = run_all_experiments_in_dir('experiments')
for experiment in experiments:
    graphs = experiment.get_figure_paths()
    for graph in graphs:
        display(Image(filename=graph))

## Option 2: Run Specific Experiments

In [ ]:
from ane_research.utils.experiments import run_experiment

# Edit as necessary - These are the 8 runs we performed in the paper
experiments_list = [
    'experiments/sst_tanh_sparsemax.jsonnet',
    'experiments/sst_tanh_softmax.jsonnet',
    'experiments/sst_sdp_sparsemax.jsonnet',
    'experiments/sst_sdp_softmax_jsonnet',
    'experiments/imdb_tanh_sparsemax.jsonnet',
    'experiments/imdb_tanh_softmax.jsonnet',
    'experiments/imdb_sdp_sparsemax.jsonnet',
    'experiments/imdb_sdp_softmax_jsonnet',
]

for experiment_file in experiments_list:
    experiment = run_experiment(experiment_file)
    graphs = experiment.get_figure_paths()
    for graph in graphs:
        # These graphs are big!
        display(Image(filename=graph))
        